I started to learn about machine learning few months ago.
Thus, this notebook may have some wrong points.
If you find these or some advices, please comment to tell me; it will help me to learn data science.

---

I used Vision Transformer at keras.

This is article.
[An Image is Worth 16x16 Words: Transformes For Image Recognition at Scale. ](https://arxiv.org/pdf/2010.11929.pdf)

I used [vit-keras](https://github.com/faustomorales/vit-keras) library to use that.

<br />

For submit, I can't connect internet, so I save vit_b16 and use from dataset.
```
base_model = vit.vit_b16(
    image_size=IMG_SIZE,
    activation='sigmoid',
    pretrained=True,
    include_top=False,
    pretrained_top=False,
)
base_model.save_model('base_model.h5')
```

This predictions are made by ensemble of 3 Transfer Learning predictions and 3 Fine tuning predictions.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import gc
import sys
import datetime

import pickle

import subprocess
import json
import datetime

from sklearn.model_selection import StratifiedKFold, train_test_split, RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras

import tensorflow.experimental.numpy as tnp

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow_addons as tfa

import seaborn as sns
from scipy.optimize import minimize


In [ ]:
# !pip install vit-keras
!pip install ../input/vit-keras/validators-0.18.2-py3-none-any.whl
!pip install ../input/vit-keras/vit_keras-0.1.0-py3-none-any.whl

In [ ]:
from vit_keras import vit, utils

In [ ]:
# Constants
IMG_SIZE = 384
CHANNELS = 3
BATCH_SIZE = 32
Q = 30
EPOCHS = 17
REPETE_NUMBER = 1
LR = 0.001
WD = 0.0001
EARRY_STOP = 5

F_EPOCHS = 4
F_BATCH_SIZE = 8
F_LR = 0.000002
F_WD = 0.0000002


DATA_DIR = '../input/petfinder-pawpularity-score'

AUTOTUNE = tf.data.experimental.AUTOTUNE  

In [ ]:
# prepare train and test
TRAIN_FILENAMES = tf.io.gfile.glob(DATA_DIR + '/train/*')
TEST_FILENAMES = tf.io.gfile.glob(DATA_DIR + '/test/*')

TRAIN_FILENAMES.sort()
TEST_FILENAMES.sort()

train_df=pd.read_csv(f'{DATA_DIR}/train.csv', usecols=['Id', 'Pawpularity'])
test_df=pd.read_csv(f'{DATA_DIR}/test.csv', usecols=['Id'])

train_df['file_path'] = TRAIN_FILENAMES
test_df['file_path'] = TEST_FILENAMES

train_df['target_value'] = train_df['Pawpularity']
train_df['stratify_label'] = pd.qcut(train_df['Pawpularity'], q = Q, labels = range(Q))

In [ ]:
confirm_index = []
with open('../input/val-index/val_index.pkl', 'rb') as web:
    confirm_index = pickle.load(web)
train_confirm = train_df.iloc[confirm_index]
train_df = train_df.drop(confirm_index)

In [ ]:
# https://www.kaggle.com/rsmits/effnet-b2-feature-models-catboost#SET-TPU-/-GPU

def build_augmenter(is_labelled):
    def augment(img):
        # Only use basic augmentations...too much augmentation hurts performance
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.95, 1.05)
        img = tf.image.random_brightness(img, 0.05)
        img = tf.image.random_contrast(img, 0.95, 1.05)
        img = tf.image.random_hue(img, 0.05)
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if is_labelled else augment

def build_decoder(is_labelled):
    def decode(path):
        # Read Image
        file_bytes = tf.io.read_file(path)
        img = tf.image.decode_jpeg(file_bytes, channels = CHANNELS)
        
        # Normalize and Resize
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        
        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if is_labelled else decode

def create_dataset(df, batch_size = BATCH_SIZE, is_labelled = False, augment = False, repeat = False, shuffle = False):
    decode_fn = build_decoder(is_labelled)
    augmenter_fn = build_augmenter(is_labelled)
    
    # Create Dataset
    if is_labelled:
        dataset = tf.data.Dataset.from_tensor_slices((df['file_path'].values, df['target_value'].values))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((df['file_path'].values))
    dataset = dataset.map(decode_fn, num_parallel_calls = AUTOTUNE)
    dataset = dataset.map(augmenter_fn, num_parallel_calls = AUTOTUNE) if augment else dataset
    dataset = dataset.repeat() if repeat else dataset
    dataset = dataset.shuffle(1024, reshuffle_each_iteration = True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    
    return dataset

In [ ]:
# Set Callbacks
def model_checkpoint(kind, fold):
    return ModelCheckpoint(f'{kind}{fold:03}.h5',
              verbose = 1, 
              monitor = 'val_root_mean_squared_error', 
              mode = 'min', 
              save_weights_only=False,
              save_best_only = True)
def early_stopping():
    return EarlyStopping(
        monitor='val_root_mean_squared_error',
        min_delta=0.0,
        patience=EARRY_STOP,
    )

In [ ]:
# OOF RMSE Placeholder
all_val_rmse = []

# Stratified Training
# kfold = StratifiedKFold(n_splits = FEATURE_FOLDS, shuffle = True, random_state = SEED)
kfold = RepeatedStratifiedKFold(n_splits = 5, n_repeats=REPETE_NUMBER)

for fold, (train_index, val_index) in enumerate(kfold.split(train_df.index, train_df['stratify_label'])):
    print(f'\n===== Fold {fold}\n')

    # Pre model.fit cleanup
    tf.keras.backend.clear_session()
    gc.collect()

    # Create Model
    base_model = keras.models.load_model('../input/nb013-v005-output/base_model.h5')
    base_model.trainable=False
    
    x = base_model.output

    prediction = tf.keras.layers.Dense(128, activation=tfa.activations.rrelu)(x)
    prediction = tf.keras.layers.Dense(1, activation=tfa.activations.rrelu)(prediction)
    
    model = keras.Model(inputs = base_model.input, outputs = prediction)

    model.compile(
        optimizer = tfa.optimizers.AdamW(
            learning_rate=LR, weight_decay=WD
        ),
        loss = tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )

    # Create TF Datasets
    trn = train_df.iloc[train_index]
    val = train_df.iloc[val_index]
    
    # training_dataset = create_dataset(small_trn, batch_size = BATCH_SIZE, is_labelled = True, augment = True, repeat = True, shuffle = True)
    training_dataset = create_dataset(trn, batch_size = BATCH_SIZE, is_labelled = True, augment = True, repeat = True, shuffle = True)
    validation_dataset = create_dataset(val, batch_size = BATCH_SIZE, is_labelled = True, augment = False, repeat = True, shuffle = False)

    # Transfer Learning
    model.fit(training_dataset,
      epochs = EPOCHS,
      steps_per_epoch = trn.shape[0] // BATCH_SIZE,
      validation_steps = val.shape[0] // BATCH_SIZE,
      callbacks = [model_checkpoint("t", fold), early_stopping()],
      validation_data = validation_dataset,
      verbose = 1
    )

    # Fine tuning
    training_dataset = create_dataset(trn, batch_size = F_BATCH_SIZE, is_labelled = True, augment = True, repeat = True, shuffle = True)
    model.load_weights(f't{fold:03}.h5')
    model.trainable = True
    model.compile(
        optimizer = tfa.optimizers.AdamW(
            learning_rate=F_LR, weight_decay=F_WD
        ),
        loss = tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    
    history = model.fit(training_dataset,
      epochs = F_EPOCHS,
      steps_per_epoch = trn.shape[0] // F_BATCH_SIZE,
      validation_steps = val.shape[0] // F_BATCH_SIZE,
      callbacks = [model_checkpoint("f", fold), early_stopping()],
      validation_data = validation_dataset,
      verbose = 1
    )

    # Validation Information
    best_val_rmse = min(history.history['val_root_mean_squared_error'])
    all_val_rmse.append(best_val_rmse)
    print(f'\nValidation RMSE: {best_val_rmse}\n')

# Summary
print(f'Final Mean RMSE for {REPETE_NUMBER*5} Fold CV Training: {np.mean(all_val_rmse)}')

In [ ]:
for file_name in os.listdir(f'./'):
    if 't' in file_name and '.h5' in file_name:
        model.load_weights(f'{file_name}')
        head_input = keras.layers.Input(model.layers[-2].input_shape[1:])
        head_model = head_input

        for layer in model.layers[-2:]:
            head_model = layer(head_model)

        head_model = keras.Model(inputs=head_input, outputs=head_model)
        head_model.compile(
            optimizer = tfa.optimizers.AdamW(
                learning_rate=F_LR, weight_decay=F_WD
            ),
            loss = tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )

        head_model.save(f'h{file_name}')

In [ ]:
confirm_dataset = create_dataset(train_confirm, batch_size = BATCH_SIZE, is_labelled = True, augment = False, repeat = False, shuffle = False)
predictions = []

In [ ]:
base_model = keras.models.load_model('../input/nb013-v005-output/base_model.h5')
base_output = base_model.predict(confirm_dataset)

In [ ]:
head_models = []
for i in range(REPETE_NUMBER*5):
    head_models.append(keras.models.load_model(f'ht{i:03}.h5'))
for head_model in head_models:
    predictions.append(np.array(head_model.predict(base_output)))

In [ ]:
trained_models = []
for i in range(REPETE_NUMBER*5):
    trained_models.append(keras.models.load_model(f'f{i:03}.h5'))
for trained_model in trained_models:
    predictions.append(np.array(trained_model.predict(confirm_dataset)))

In [ ]:
for i, prediction in enumerate(predictions):
    print(i, prediction.mean(), prediction.std(), np.sqrt(mean_squared_error(train_confirm['Pawpularity'], prediction)))

In [ ]:
# For measuring similarity
score_table = []
for prediction1 in predictions:
    temp_line = []
    for prediction2 in predictions:
        temp_line.append(np.sqrt(mean_squared_error(prediction1, prediction2)))
    score_table.append(temp_line)
sns.heatmap(score_table)

Fine tuning may get more good grade. Thus, I set high ratio of ensemble.

Transfer learning ones (ht) 4 : Fine tuning ones (f) 6

In [ ]:
ratio_bet_htf = [0.4, 0.6]
best_ratios = []
for one_ratio in ratio_bet_htf:
    for i in range(REPETE_NUMBER*5):
        best_ratios.append(1.0 * one_ratio / (REPETE_NUMBER * 5))

In [ ]:
best_ratios

In [ ]:
y_pred = np.zeros((train_confirm.shape[0], 1))
for i, ratio in enumerate(best_ratios):
    y_pred += (predictions[i] * ratio)
print(y_pred.mean(), y_pred.std(), np.sqrt(mean_squared_error(train_confirm['Pawpularity'], y_pred)))

In [ ]:
sub_dataset = create_dataset(test_df, batch_size = BATCH_SIZE, is_labelled = False, augment = False, repeat = False, shuffle = False)
predictions = []

In [ ]:
base_output = base_model.predict(sub_dataset)
for head_model in head_models:
    predictions.append(np.array(head_model.predict(base_output)))

In [ ]:
for trained_model in trained_models:
    predictions.append(np.array(trained_model.predict(sub_dataset)))

In [ ]:
y_pred = np.zeros((test_df.shape[0], 1))

for i, ratio in enumerate(best_ratios):
    y_pred += (predictions[i] * ratio)
print(y_pred.mean(), y_pred.std())

In [ ]:
test_df['Pawpularity'] = y_pred
test_df = test_df[["Id", "Pawpularity"]]
test_df.to_csv("submission.csv", index=False)

In [ ]:
test_df